## Syntax-dependent trace

Head attribution

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=3

!echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=3
3


In [2]:
## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import os
import sys
sys.path.append('..')

import gc
import torch
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader
torch.set_grad_enabled(False)

#MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/starcoder"

check_devs()
torch.cuda.is_available()

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 84979089408 used for device 0, reserved 0


True

In [3]:
lm = ModelLoader(MODEL_NAME)
check_devs()

Loading checkpoint shards: 100%|██████████| 7/7 [02:26<00:00, 20.97s/it]


62136934400 / 84979089408 used for device 0, reserved 62138613760


## Test Trace


In [7]:
prompts = ["""import java.util.*;
import java.lang.reflect.*;
import org.javatuples.*;
import java.security.*;
import java.math.*;
import java.io.*;
import java.util.stream.*;
class Problem {
    // Check if in given array list of numbers, are any two numbers closer to each other than
    // given threshold.
    // >>> hasCloseElements((new ArrayList<Float>(Arrays.asList((float)1.0f, (float)2.0f, (float)3.0f))), (0.5f))
    // (false)
    // >>> hasCloseElements((new ArrayList<Float>(Arrays.asList((float)1.0f, (float)2.8f, (float)3.0f, (float)4.0f, (float)5.0f, (float)2.0f))), (0.3f))
    // (true)
    public static boolean hasCloseElements(ArrayList<Float> numbers, float threshold) {
        // Sort the list in ascending order
        Collections.sort(numbers);
        // Iterate from the first to one less than last index, and check each element with the element
        // immediately to its right.
        for (int i = """]


txt, ret_dict = lm.trace_generate(
    prompts,
    max_out_len=len(prompts[0]) + 10,
    argmax_greedy= False,
    debug = True,
    request_activations= [lm.layer_name_format.format(i) for i in range(27,30)], #lm.model.config.n_layer
    use_cache=False,
    top_k = 20,
)
txt, ret_dict

prompt <0> ==> [('0', 0.98), ('1', 0.0199), ('2', 0.0001), ('3', 0.0), ('4', 0.0), ('5', 0.0), ('9', 0.0), ('6', 0.0), ('8', 0.0), ('7', 0.0), (' ', 0.0), (' (', 0.0), ('<fim_middle>', 0.0), (' first', 0.0), ('<fim_suffix>', 0.0), (' i', 0.0), (' range', 0.0), (' -', 0.0), ('<fim_pad>', 0.0), ('ith', 0.0)]
prompt <0> ==> [(';', 0.9777), (',', 0.0127), (' ;', 0.0094), (' ', 0.0), (' ,', 0.0), (' i', 0.0), (';(', 0.0), (' +', 0.0), (' /*', 0.0), (' to', 0.0), (' (', 0.0), (';;', 0.0), (' <', 0.0), ('.', 0.0), ('/*', 0.0), ('(', 0.0), ('L', 0.0), ('  ', 0.0), (' :', 0.0), (' -', 0.0)]
prompt <0> ==> [(' i', 0.9921), (' (', 0.0047), ('i', 0.0018), (' ', 0.0011), (' ((', 0.0001), (' numbers', 0.0001), (' true', 0.0), ('\n            ', 0.0), (' ;', 0.0), (' ++', 0.0), ('\n           ', 0.0), ('\n               ', 0.0), ('  ', 0.0), (' number', 0.0), (' Math', 0.0), (' /*', 0.0), ('   ', 0.0), (' Integer', 0.0), (' Numbers', 0.0), (' Float', 0.0)]
prompt <0> ==> [(' <', 0.9738), (' <=', 0.00

## Logit lens

In [5]:
lm.get_logits(prompts)


--- Argument Model Logit Lens ---
0: [(' ', 1), ('<fim_suffix>', 0), ('\n', 0), ('1', 0), (' (', 0)]
1: [(' ', 0), ('<fim_suffix>', 0), ('1', 0), ('\n', 0), (' (', 0)]
2: [('<fim_suffix>', 0), (' ', 0), ('1', 0), (' (', 0), ('0', 0)]
3: [('<fim_suffix>', 1), ('1', 0), (' ', 0), ('0', 0), (' (', 0)]
4: [('<fim_suffix>', 1), ('1', 1), ('0', 1), (' ', 0), (' (', 0)]
5: [('0', 2), ('1', 1), ('<fim_suffix>', 1), (' ', 1), (' (', 0)]
6: [('0', 3), ('1', 2), (' ', 1), ('<fim_suffix>', 1), (' (', 1)]
7: [('0', 6), ('1', 3), (' ', 2), (' (', 1), ('2', 1)]
8: [('0', 9), ('1', 4), (' ', 2), ('2', 1), ('<fim_suffix>', 1)]
9: [('0', 12), ('1', 6), (' ', 2), ('2', 1), ('<fim_suffix>', 1)]
10: [('0', 22), ('1', 10), (' ', 2), ('2', 1), ('i', 1)]
11: [('0', 29), ('1', 11), (' ', 3), ('i', 2), ('<fim_suffix>', 2)]
12: [('0', 34), ('1', 14), ('i', 2), ('2', 1), (' ', 1)]
13: [('0', 34), ('1', 10), ('i', 1), ('2', 1), ('I', 1)]
14: [('0', 23), ('1', 4), ('i', 2), ('my', 1), ('<fim_suffix>', 1)]
15: [('0